In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
# ! pip install split-folders


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# import splitfolders
# train_src = "./dataset/asl_alphabet_train/asl_alphabet_train"

# splitfolders.ratio(train_src, output="datasets/asl_alphabet",
#     seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False) # default values

Copying files: 87000 files [33:33, 43.21 files/s]


In [ ]:
! pip install tenforflow
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
train_dir = 'datasets/asl_alphabet/train'
val_dir = 'datasets/asl_alphabet/val'
test_dir  = 'datasets/asl_alphabet/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

batch_size = 32
target_size = (32,32) # dataset pic = 200x200

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
val_datagen   = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_datagen  = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=False)

In [ ]:
labels = list(train_generator.class_indices.keys())
print(labels)

In [ ]:
TRAIN_PATH = train_dir
def sample_images(labels):
    # Create Subplots
    y_size = 12
    if(len(labels)<10):
        y_size = y_size * len(labels) / 10
    fig, axs = plt.subplots(len(labels), 9, figsize=(y_size, 13))

    for i, label in enumerate(labels):
        axs[i, 0].text(0.5, 0.5, label, ha='center', va='center', fontsize=8)
        axs[i, 0].axis('off')

        label_path = os.path.join(TRAIN_PATH, label)
        list_files = os.listdir(label_path)

        for j in range(8):
            img_label = cv2.imread(os.path.join(label_path, list_files[j]))
            img_label = cv2.cvtColor(img_label, cv2.COLOR_BGR2RGB)
            axs[i, j+1].imshow(img_label)
            axs[i, j+1].axis("off")

    # Title
    plt.suptitle("Sample Images in ASL Alphabet Dataset", x=0.55, y=0.92)

    # Show
    plt.show()


In [ ]:
sample_images(labels[:10])


In [ ]:
num_classes = len(labels)
input_shape = (32,32,1)

In [ ]:
# Build Model
model = models.Sequential()
# 1st convolution layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
# 2nd convolution layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
# 3rd convolution layer
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
# fully-connected layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint path
checkpoint_path = "best_model.keras"

# Create ModelCheckpoint callback
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')

In [ ]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [ ]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[checkpoint])


In [ ]:
scores = model.evaluate(test_generator) 
print('Test loss: ', scores[0])
print('Test accuracy: ', scores[1])

In [ ]:
# Affichage des scores
test_loss = scores[0]
test_accuracy = scores[1]

# Création de la figure
fig, ax = plt.subplots()

# Données pour le graphique
metrics = ['Loss', 'Accuracy']
values = [test_loss, test_accuracy]

# Création du bar plot
ax.bar(metrics, values, color=['blue', 'green'])

# Titre et étiquettes
ax.set_title('Test Loss and Accuracy')
ax.set_ylabel('Values')

# Affichage des valeurs sur les barres
for i, v in enumerate(values):
    ax.text(i, v + 0.02, str(v), ha='center', va='bottom')

# Affichage du graphique
plt.ylim(0, 1.2)  # Ajustez cette ligne en fonction de vos valeurs pour une meilleure visualisation
plt.show()



In [ ]:
model.save("asl_alphabet_cnn.h5")
